# Importing the required libraries

In [23]:
import torch
import pandas as pd
import numpy as np
import os
import warnings

from transformers import AutoTokenizer, DistilBertForSequenceClassification, AutoModelForSeq2SeqLM
from tqdm import tqdm
from torchvision import models
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from keras.preprocessing import image
from torchmetrics.classification import MultilabelF1Score

### Setting up the environment
***

In [24]:
warnings.filterwarnings("ignore")

***

# Data Preprocessing

In [25]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
genres = [genre.strip() for genre in genre_all]

mapping = {}
for genre, i in enumerate(genres):
    mapping[genre] = i

In [26]:
users = pd.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pd.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                        sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pd.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                        sep='::', names=['movieid', 'title', 'genre'], encoding='ISO-8859-1', index_col=False).set_index('movieid')
movies_test = pd.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                        sep='::', names=['movieid', 'title', 'genre'], encoding='ISO-8859-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_train.index.name = 'ID'
movies_test['genre'] = movies_test.genre.str.split('|')
movies_test.index.name = 'ID'

*** 
Must add overview column to this table via generator

In [27]:
def preprocess(df, path='ml1m/content/dataset/ml1m-images', genres=genres) -> pd.DataFrame:
    df['img_path'] = df.apply(lambda x: os.path.join(path, str(x.name) + '.jpg'), axis=1)
    df['label'] = df.genre.apply(lambda x: [1 if genre in x else 0 for genre in genres])
    df.drop(columns=['genre'], inplace=True)
    df = df.reset_index(drop=True)
    return df

***

In [28]:
trainset = preprocess(movies_train)
testset = preprocess(movies_test)
validset = testset.sample(frac=0.5, random_state=42)
testset = testset.drop(validset.index)

In [29]:
print(len(trainset), len(testset), len(validset))

3106 389 388


3883 movies loaded to trainset and testset

# Model Implementation

### Sub-models
***

In [30]:
tokenizer_title = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_title = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=18)
model_title.config.id2label = mapping

tokenizer_overview = AutoTokenizer.from_pretrained("NowT-coding/distilBert-movie-genre-classification")
model_overview = DistilBertForSequenceClassification.from_pretrained("NowT-coding/distilBert-movie-genre-classification", problem_type="multi_label_classification", num_labels=18)
model_overview.config.id2label = mapping

model_image = models.resnet101(pretrained=False)
model_image.fc = torch.nn.Linear(2048, len(genres))

tokenizer_gen = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")
model_gen = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_title.to(device)
model_overview.to(device)
model_image.to(device)
model_gen.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

### Deep Fusion Multimodal Model
***

In [31]:
class Multimodal(torch.nn.Module):
    def __init__(self, model1, model2, model3):
        super().__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.fc1 = torch.nn.Linear(18, 18)
        self.fc2 = torch.nn.Linear(18, 18)
        self.fc3 = torch.nn.Linear(18, 18)

    def forward(self, 
                title_input_ids, title_attention_mask,
                plot_input_ids, plot_attention_mask,
                image_input):
        title_output = self.model1(title_input_ids, title_attention_mask)
        plot_output = self.model2(plot_input_ids, plot_attention_mask)
        image_output = self.model3(image_input)

        title_output = self.fc1(title_output.logits)
        plot_output = self.fc2(plot_output.logits)
        image_output = self.fc3(image_output)
        
        output = torch.add(title_output, plot_output)
        output = torch.add(output, image_output)
        return output

# Custom Datasets & Data Loaders

***
### Custom Dataset
***

In [32]:
class Poroset(torch.utils.data.Dataset):
    def __init__(self, df, 
                 tokenizer1, tokenizer2, 
                 generator, tokenizer_gen,
                 max_len=256,
                 device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        self.df = df
        self.tokenizer1 = tokenizer1
        self.tokenizer2 = tokenizer2
        self.max_len = max_len
        self.image = image
        self.generator = generator
        self.tokenizer_gen = tokenizer_gen
        self.device = device
        self.transform = v2.Compose([
            v2.Resize((224, 224)),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        title = row['title']

        self.generator.eval()
        quote = 'What is the story of the movie {}?'
        input_ids = self.tokenizer_gen.encode(quote.format(title), return_tensors='pt').to(self.device)
        outputs = self.generator.generate(input_ids, max_length=256, do_sample=True, temperature=0.09)
        plot = self.tokenizer_gen.decode(outputs[0], skip_special_tokens=True)

        label = row['label']
        title_encoding = self.tokenizer1(title, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        plot_encoding = self.tokenizer2(plot, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        path='ml1m/content/dataset/ml1m-images'
        image_path = os.path.join(path, str(row.name) + '.jpg')
        if os.path.exists(image_path):
            image_input = image.load_img(image_path)
            image_input = self.transform(image_input)
        else:
            image_input = torch.zeros((3, 224, 224))
        
        return {
            'title': title,
            'plot': plot,
            'title_input_ids': title_encoding['input_ids'].squeeze(),
            'title_attention_mask': title_encoding['attention_mask'].squeeze(),
            'plot_input_ids': plot_encoding['input_ids'].squeeze(),
            'plot_attention_mask': plot_encoding['attention_mask'].squeeze(),
            'image_input': image_input,
            'label': torch.FloatTensor(label)
        }

In [33]:
trainset = Poroset(trainset, tokenizer_title, tokenizer_overview, model_gen, tokenizer_gen)
validset = Poroset(validset, tokenizer_title, tokenizer_overview, model_gen, tokenizer_gen)
testset = Poroset(testset, tokenizer_title, tokenizer_overview, model_gen, tokenizer_gen)

***
### Custom Data Loader
***

In [34]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

Check if the data loader is working properly

In [35]:
sample = next(iter(trainloader))

# First sample of the batch
print('Title: ', sample['title'][0])
print('Plot: ', sample['plot'][0])
print('Label: ', sample['label'][0])
print('Image: ', sample['image_input'][0].shape)

Title:  Zachariah (1971)
Plot:  Zachariah is a 1971 American drama film about a young woman named Isabelle who is diagnosed with cancer and is struggling to make ends meet. She is a successful businesswoman who is determined to help her family and friends through her journey. However, she is also a troublemaker and is forced to confront her own mortality.
Label:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
Image:  torch.Size([3, 224, 224])


# Setting up the Trainer

***
### GPU & Model Configuration
***

In [36]:
model = Multimodal(model_title, model_overview, model_image)
model.to(device)

for param in model.model2.parameters():
    param.requires_grad = False
    
model

Multimodal(
  (model1): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn

***
### Setting up loss function & optimizer
***

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

***
### Trainer & Validation
***

In [ ]:
def train(epoch):
    model.train()
    for _, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        title_input_ids = data['title_input_ids'].to(device)
        title_attention_mask = data['title_attention_mask'].to(device)
        plot_input_ids = data['plot_input_ids'].to(device)
        plot_attention_mask = data['plot_attention_mask'].to(device)
        image_input = data['image_input'].to(device)
        label = data['label'].to(device)

        optimizer.zero_grad()
        outputs = model(
            title_input_ids, title_attention_mask,
            plot_input_ids, plot_attention_mask,
            image_input
        )
        
        loss = loss_fn(outputs, label)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, Train Loss: {loss.item()}')

In [ ]:
def valid(epoch):
    model.eval()
    with torch.no_grad():
        for _, data in tqdm(enumerate(validloader, 0), total=len(validloader)):
            title_input_ids = data['title_input_ids'].to(device)
            title_attention_mask = data['title_attention_mask'].to(device)
            plot_input_ids = data['plot_input_ids'].to(device)
            plot_attention_mask = data['plot_attention_mask'].to(device)
            image_input = data['image_input'].to(device)
            label = data['label'].to(device)

            outputs = model(
                title_input_ids, title_attention_mask,
                plot_input_ids, plot_attention_mask,
                image_input
            )

            loss = loss_fn(outputs, label)
    print(f'Epoch: {epoch}, Valid Loss: {loss.item()}')

# Training & Validating Loop

In [ ]:
for epoch in range(10):
    train(epoch)
    valid(epoch)

# Testing the model

In [ ]:
# Validation
def test(testing_loader):
    model.eval()
    fin_targets = []
    fin_outputs = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader), total=len(testing_loader)):
            title_input_ids = data['title_input_ids'].to(device)
            title_attention_mask = data['title_attention_mask'].to(device)
            plot_input_ids = data['plot_input_ids'].to(device)
            plot_attention_mask = data['plot_attention_mask'].to(device)
            image_input = data['image_input'].to(device)
            label = data['label'].to(device)

            outputs = model(
                title_input_ids, title_attention_mask,
                plot_input_ids, plot_attention_mask,
                image_input
            )

            fin_targets.extend(label.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [ ]:
outputs, targets = test(testloader)

outputs = np.array(outputs) >= 0.5

In [ ]:
# Multi-label F1 score, macro-averaged
from torchmetrics.classification import MultilabelF1Score
f1 = MultilabelF1Score(num_labels=18, threshold=0.5, average='macro')
f1.update(torch.tensor(outputs), torch.tensor(targets))

f1.compute()

In [ ]:
# Save model
torch.save(model.state_dict(), 'model.pt')

In [ ]:
# Map@k score
def mapk_score(y_true, y_pred):
    score = 0.0
    for k in range(1, 19):
        score += np.sum(y_pred[:, :k] == y_true) / k
    score /= y_true.shape[0]
    return score

In [ ]:
# Inferencing title and overview
def inference(title, overview = None, genres = genres):
    model.eval()
    with torch.no_grad():
        title_encoding = tokenizer(title, truncation=True, padding='max_length', max_length=256, return_tensors='pt')
        title_input_ids = title_encoding['input_ids'].to(device)
        title_attention_mask = title_encoding['attention_mask'].to(device)
        if overview is not None:
            overview_encoding = tokenizer(overview, truncation=True, padding='max_length', max_length=256, return_tensors='pt')
            overview_input_ids = overview_encoding['input_ids'].to(device)
            overview_attention_mask = overview_encoding['attention_mask'].to(device)
            outputs = model(title_input_ids, title_attention_mask, overview_input_ids, overview_attention_mask)
        else:
            outputs = model(title_input_ids, title_attention_mask, None, None)
        outputs = outputs.cpu().detach().numpy().tolist()
        outputs = np.array(outputs) >= 0.5
        print([genres[i] for i in range(len(genres)) if outputs[0][i] == 1])

In [ ]:
print(testset[3]['title'])
#print genres of the movie in testset in word
for i in range(len(genres)):
    if testset[3]['label'][i] == 1:
        print(genres[i])

In [ ]:
inference(testset[3]['title'],testset[3]['overview'])